# Exercises Chapter 6 - Decision Trees

## Exercise 1

What is the approximate depth of a Decision Tree trained (without restrictions)
on a training set with one million instances?

$\log_2(1000000)$

In [3]:
import numpy as np
np.log2(1000000)

19.931568569324174

## Exercise 2

Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?

The Gini impurity decreases at each level

## Exercise 3

If a Decision Tree is overfitting the training set, is it a good idea to try decreasing
max_depth ?

Decrease max_depth to provide more regularization.

## Exercise 4

If a Decision Tree is underfitting the training set, is it a good idea to try scaling
the input features?

Scaling has no relevance for Decision Trees

## Exercise 5

If it takes one hour to train a Decision Tree on a training set containing 1 million
instances, roughly how much time will it take to train another Decision Tree on a
training set containing 10 million instances?

???

## Exercise 6

If your training set contains 100,000 instances, will setting presort=True speed
up training?

Probably not, as pre-sorting will take longer than fitting directly

## Exercise 7

Train and fine-tune a Decision Tree for the moons dataset by following these
steps:

a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

b. Use train_test_split() to split the dataset into a training set and a test set.

c. Use grid search with cross-validation (with the help of the GridSearchCV
class) to find good hyperparameter values for a DecisionTreeClassifier .
Hint: try various values for max_leaf_nodes .

d. Train it on the full training set using these hyperparameters, and measure
your model’s performance on the test set. You should get roughly 85% to 87%
accuracy.

In [1]:
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [2]:
X, y = make_moons(n_samples=10000, noise=0.4)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
param_grid = {'max_leaf_nodes':[3, 4, 5, 6, 10],
              'max_depth':[None, 5, 10], 
              'min_samples_leaf':[1,2, 3,5,10],
              'min_samples_split':[2, 3, 4, 8]}

In [5]:
clf = DecisionTreeClassifier()

In [6]:
grid = GridSearchCV(clf, param_grid, n_jobs=-1)

In [7]:
_ = grid.fit(X_train, y_train)

In [8]:
grid.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=4,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [9]:
y_pred = grid.predict(X_test)

In [10]:
print(accuracy_score(y_pred, y_test))

0.854


## Exercise 8

Grow a forest by following these steps:

a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.

b. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [18]:
import numpy as np

from scipy.stats import mode

from sklearn.base import clone
from sklearn.model_selection import ShuffleSplit

In [14]:
shuffler = ShuffleSplit(n_splits=1000, train_size=100, test_size=1)

In [15]:
trees = []
for idx in shuffler.split(X_train):
    trees.append(clone(grid.best_estimator_))
    trees[-1].fit(X_train[idx[0],],y_train[idx[0],])

In [16]:
scores = []
for tree in trees:
    y_pred = tree.predict(X_test)
    scores.append(accuracy_score(y_pred, y_test))

In [22]:
print('Average accuracy score is {:6.4f} with a standard deviation of {:6.4f}'.format(np.mean(scores), np.std(scores)))

Average score is 0.8244 with a standard deviation of 0.0259


In [23]:
predictions = np.zeros((1000, len(y_test)))
for i,tree in enumerate(trees):
    predictions[i,:] = tree.predict(X_test)

In [25]:
preds_rf = np.apply_along_axis(mode, 0, predictions)
y_pred_rf = preds_rf[0,:,:].squeeze()

In [27]:
print('Accuracy Score of Random Forest: {:6.4f}'.format(accuracy_score(y_pred_rf, y_test)))

Accuracy Score of Random Forest: 0.8555
